In [25]:
import os
import json

from config import GEMINI_API_KEY

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from pprint import pprint

In [12]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

In [13]:
theme_prompt = PromptTemplate(
    input_variables=[],
    template="""
Generate a JSON object containing 5-10 key themes or points of interest that prospective university students commonly inquire about when considering a university experience. The keys of the JSON object should be sequential integers starting from 0, and the values should be descriptive strings representing the themes.

Example of the desired output format:
{{
  "0": "Student Life",
  "1": "Extracurriculars",
  "2": "Academics",
  "3": "Admissions"
}}

The output should be in a JSON-like format within the curly braces.
    """
)

In [18]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

json_output_parser = JsonOutputParser()

chain = llm | json_output_parser

In [21]:
theme_prompt_results = chain.invoke(theme_prompt.format())

In [24]:
pprint(theme_prompt_results)

{'0': 'Academic Programs and Faculty Quality',
 '1': 'Career Services and Internship Opportunities',
 '2': 'Campus Life and Student Culture',
 '3': 'Tuition, Financial Aid, and Scholarships',
 '4': 'Location, Safety, and Accommodation',
 '5': 'Research Opportunities (for undergraduate/graduate students)',
 '6': 'Student Support Services (e.g., counseling, disability services)',
 '7': 'Diversity and Inclusion Initiatives'}


In [34]:
theme_prompt_results = {
    '0': 'Academic Programs and Faculty',
    '1': 'Career Services and Internship Opportunities',
    '2': 'Campus Life and Student Culture',
    '3': 'Tuition, Financial Aid, and Scholarships',
    '4': 'Location, Safety, and Accommodation',
    '5': 'Research Opportunities (for undergraduate/graduate students)',
    '6': 'Student Support Services (e.g., counseling, disability services)',
}

In [35]:
question_prompt_template = PromptTemplate(
    input_variables=["THEME"],
    template="""
You are helping to generate lexicon-rich question templates for prospective students exploring the Singapore University of Technology and Design (SUTD). Each question should contain a lexicon placeholder (e.g., {{CLUB}}, {{PROGRAM_TYPE}}, {{SEMESTER_YEAR}}) that can later be substituted with concrete values retrieved from the web.

Here are some themes relevant to prospective students, along with sample question templates:

Theme: Student Life and Culture  
{{
  "Social Events": "What kind of social events or traditions does SUTD host during {{SEMESTER_OR_PERIOD}}?",
  "Campus Culture": "What is student culture like for those studying in {{PROGRAM_TYPE}} at SUTD?",
  "Arts and Culture": "What opportunities are there for students interested in {{ARTS_ACTIVITY}} at SUTD?"
}}

Theme: Admissions and Applications  
{{
  "Application Requirements": "What are the application requirements for {{PROGRAM_TYPE}} programs at SUTD?",
  "Essay Guidance": "Do you have any advice for writing the application essay for {{PROGRAM_TYPE}} at SUTD?",
  "Standardized Tests": "What are the policies regarding {{TEST_TYPE}} scores when applying to SUTD?",
  "Application Deadline": "What is the application deadline for entry in {{SEMESTER_YEAR}}?"
}}

Theme: {THEME}
{{
  "{{SUB_THEME_1_KEY}}": "{{SUB_THEME_1_QUERY}}",
  "{{SUB_THEME_2_KEY}}": "{{SUB_THEME_2_QUERY}}",
  "{{SUB_THEME_3_KEY}}": "{{SUB_THEME_3_QUERY}}",
  "{{SUB_THEME_4_KEY}}": "{{SUB_THEME_4_QUERY}}"
}}
Generate 3-5 relevant sub-themes (keys) and corresponding question templates (values) for the theme: {THEME}. Use placeholders like {{COURSE}}, {{FACILITY}}, {{HOSTEL}}, {{CLUB}}, {{SEMESTER_YEAR}}, etc., to keep the questions lexicon-ready. The output should be in a JSON-like format within the curly braces.
"""
)

question_templates = {}
for theme in theme_prompt_results.values():
    question_template_prompt_results = chain.invoke(question_prompt_template.format(THEME=theme))
    question_templates[theme] = question_template_prompt_results

In [ ]:
for theme in question_templates.keys():
    for subtheme, question_template in question_templates[theme].items():
        print(question_template)

What specific topics are covered in the {COURSE} course at SUTD?
What are the current research interests of faculty in the {DEPARTMENT} department at SUTD?
What learning resources, such as {FACILITY} or software, are available to students in the {PROGRAM_TYPE} program?
What are some examples of recent capstone projects completed by students in the {PROGRAM_TYPE} program?
What kind of industry connections or partnerships does SUTD have related to the {DEPARTMENT} department?
Does SUTD host career fairs focused on {INDUSTRY} or {PROGRAM_TYPE} students?
What kind of support does SUTD offer to students seeking internships in {COUNTRY} or at {COMPANY_TYPE}?
How can the SUTD alumni network help students with their career prospects in {FIELD_OF_STUDY}?
Are there resume writing workshops or career counseling services available for {PROGRAM_TYPE} students at SUTD?
What is the job placement rate for SUTD graduates within {MONTHS_AFTER_GRADUATION} of graduation, specifically for {PROGRAM_TYPE}?
W